In [14]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


from traditional_model import *


### Class for the Neural Network

In [17]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [18]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [19]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [20]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Using best configuration for the congressional voting dataset

#### Loading the dataset

In [21]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


### Without CV

In [22]:
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [35]:
input_size = train_X.shape[1]
num_classes = 2
learning_rate = 0.01
batch_size = 64
num_epochs = 100
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/100, Average Loss: 0.8204039831956228
Epoch 2/100, Average Loss: 0.6987561782201132
Epoch 3/100, Average Loss: 0.6630964974562327
Epoch 4/100, Average Loss: 0.5736258924007416
Epoch 5/100, Average Loss: 0.4424207905928294
Epoch 6/100, Average Loss: 0.3960420439640681
Epoch 7/100, Average Loss: 0.29747725029786426
Epoch 8/100, Average Loss: 0.2279346063733101
Epoch 9/100, Average Loss: 0.22333091124892235
Epoch 10/100, Average Loss: 0.3738237793246905
Epoch 11/100, Average Loss: 0.2867339824636777
Epoch 12/100, Average Loss: 0.14389140841861567
Epoch 13/100, Average Loss: 0.13733384509881338
Epoch 14/100, Average Loss: 0.09313429457445939
Epoch 15/100, Average Loss: 0.10354475133741896
Epoch 16/100, Average Loss: 0.10233023607482512
Epoch 17/100, Average Loss: 0.08094628248363733
Epoch 18/100, Average Loss: 0.07607210986316204
Epoch 19/100, Average Loss: 0.06412325814987223
Epoch 20/100, Average Loss: 0.06259010794262092
Epoch 21/100, Average Loss: 0.05415845879664024
Epoch 22/1

### With CV

In [36]:
X = cong_voting.drop('class', axis=1).values
y = cong_voting['class'].values

num_splits = 5

kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []

for train_index, test_index in kf.split(X, y):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    train_X_fold = torch.tensor(X_train, dtype=torch.float32)
    train_Y_fold = torch.tensor(y_train, dtype=torch.long)
    test_X_fold = torch.tensor(X_test, dtype=torch.float32)
    test_Y_fold = torch.tensor(y_test, dtype=torch.long)

    train_dataset_fold = TensorDataset(train_X_fold, train_Y_fold)
    train_loader_fold = DataLoader(train_dataset_fold, batch_size=batch_size, shuffle=True)

    test_dataset_fold = TensorDataset(test_X_fold, test_Y_fold)
    test_loader_fold = DataLoader(test_dataset_fold, batch_size=batch_size, shuffle=False)

    model_fold = NN(input_size=input_size, num_classes=num_classes,
                    hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)
    criterion_fold = nn.CrossEntropyLoss()
    optimizer_fold = optim.Adam(model_fold.parameters(), lr=learning_rate)

    train_model(model_fold, train_loader_fold, optimizer_fold, criterion_fold, num_epochs)


    train_accuracy_fold = check_accuracy(train_loader_fold, model_fold)
    test_accuracy_fold = check_accuracy(test_loader_fold, model_fold)

    train_accuracies.append(train_accuracy_fold)
    test_accuracies.append(test_accuracy_fold)

print(f"Average accuracy on training set: {np.mean(train_accuracies)}")
print(f"Average accuracy on test set: {np.mean(test_accuracies)}")

Epoch 1/100, Average Loss: 0.8180125951766968
Epoch 2/100, Average Loss: 0.8117269078890482
Epoch 3/100, Average Loss: 0.6783948938051859
Epoch 4/100, Average Loss: 0.6918448209762573
Epoch 5/100, Average Loss: 0.5437196294466654
Epoch 6/100, Average Loss: 0.48232629895210266
Epoch 7/100, Average Loss: 0.3559349775314331
Epoch 8/100, Average Loss: 0.30739949146906537
Epoch 9/100, Average Loss: 0.2549964189529419
Epoch 10/100, Average Loss: 0.2912028183539708
Epoch 11/100, Average Loss: 0.19942163427670798
Epoch 12/100, Average Loss: 0.18456491082906723
Epoch 13/100, Average Loss: 0.17688415199518204
Epoch 14/100, Average Loss: 0.1865780552228292
Epoch 15/100, Average Loss: 0.13970640301704407
Epoch 16/100, Average Loss: 0.18340398743748665
Epoch 17/100, Average Loss: 0.20670132835706076
Epoch 18/100, Average Loss: 0.3920699805021286
Epoch 19/100, Average Loss: 0.19739287594954172
Epoch 20/100, Average Loss: 0.13955268015464148
Epoch 21/100, Average Loss: 0.17436438302199045
Epoch 22/10

Epoch 98/100, Average Loss: 0.01198496591920654
Epoch 99/100, Average Loss: 0.022237518802285194
Epoch 100/100, Average Loss: 0.048407101382811867
Epoch 1/100, Average Loss: 0.8420432408650717
Epoch 2/100, Average Loss: 0.7978150049845377
Epoch 3/100, Average Loss: 0.6780501206715902
Epoch 4/100, Average Loss: 0.7014119227727255
Epoch 5/100, Average Loss: 0.638034462928772
Epoch 6/100, Average Loss: 0.521091361840566
Epoch 7/100, Average Loss: 0.41388893127441406
Epoch 8/100, Average Loss: 0.36143534382184345
Epoch 9/100, Average Loss: 0.2692701866229375
Epoch 10/100, Average Loss: 0.19202392796675363
Epoch 11/100, Average Loss: 0.16828558842341104
Epoch 12/100, Average Loss: 0.15494502087434134
Epoch 13/100, Average Loss: 0.23360012471675873
Epoch 14/100, Average Loss: 0.2103150337934494
Epoch 15/100, Average Loss: 0.1641582747300466
Epoch 16/100, Average Loss: 0.1669037640094757
Epoch 17/100, Average Loss: 0.16539868215719858
Epoch 18/100, Average Loss: 0.16679110998908678
Epoch 19/1

Epoch 94/100, Average Loss: 0.03844649841388067
Epoch 95/100, Average Loss: 0.041466968754927315
Epoch 96/100, Average Loss: 0.03610756527632475
Epoch 97/100, Average Loss: 0.049347108229994774
Epoch 98/100, Average Loss: 0.04864650623251995
Epoch 99/100, Average Loss: 0.03906491786862413
Epoch 100/100, Average Loss: 0.04855617849777142
Epoch 1/100, Average Loss: 0.8526902596155802
Epoch 2/100, Average Loss: 0.8033377925554911
Epoch 3/100, Average Loss: 0.6664307514826456
Epoch 4/100, Average Loss: 0.6913187106450399
Epoch 5/100, Average Loss: 0.6324726939201355
Epoch 6/100, Average Loss: 0.6088979442914327
Epoch 7/100, Average Loss: 0.5952257513999939
Epoch 8/100, Average Loss: 0.46835898359616596
Epoch 9/100, Average Loss: 0.38707205653190613
Epoch 10/100, Average Loss: 0.32711730897426605
Epoch 11/100, Average Loss: 0.2868749449650447
Epoch 12/100, Average Loss: 0.23903718094031015
Epoch 13/100, Average Loss: 0.18091660737991333
Epoch 14/100, Average Loss: 0.18090970814228058
Epoch 

### MLP Classifier

In [37]:
learning_rate = 0.01
batch_size = 64
num_epochs = 100
hidden_layer_sizes = (25, 30)
activation_function = 'tanh'

num_splits = 5

kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []

for train_index, test_index in kf.split(X, y):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # MLPClassifier
    mlp_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                              activation=activation_function,
                              learning_rate_init=learning_rate,
                              batch_size=batch_size,
                              max_iter=num_epochs)
    
    # Train MLPClassifier
    mlp_model.fit(X_train, y_train)

    # Make predictions on the test set
    mlp_predictions_train = mlp_model.predict(X_train)
    mlp_predictions_test = mlp_model.predict(X_test)

    # Calculate accuracy
    train_accuracy_fold = accuracy_score(y_train, mlp_predictions_train)
    test_accuracy_fold = accuracy_score(y_test, mlp_predictions_test)

    train_accuracies.append(train_accuracy_fold)
    test_accuracies.append(test_accuracy_fold)

print(f"Average accuracy on training set: {np.mean(train_accuracies)}")
print(f"Average accuracy on test set: {np.mean(test_accuracies)}")

Average accuracy on training set: 0.9666002258986113
Average accuracy on test set: 0.9677589852008456
